In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

In [2]:
from keras.models import load_model
model = load_model('chatbotmodel.h5')
import json
import random
#loading the json file
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

Using TensorFlow backend.


In [3]:
def clean_up_sentence(sentence):
    # tokenizing - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [4]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [5]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [6]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [7]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

Creating GUI

In [8]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

In [9]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

In [10]:
base = Tk()
base.title("Grasp Coding | Ecommerce Chatbot")
# base.geometry("400x500")
# base.resizable(width=FALSE, height=FALSE)
# x = (base.winfo_screenwidth() - base.winfo_reqwidth()) / 2
# y = (base.winfo_screenheight() - base.winfo_reqheight()) / 2
# base.geometry("+%d+%d" % (x, y))

width = base.winfo_screenwidth()
height = base.winfo_screenheight()
x = int(width / 2 - 600 / 2)
y = int(height / 2 - 400 / 2)
str1 = "390x480+" + str(x) + "+" + str(y)
base.geometry(str1)

footer = Label(text="© 2020 Grasp Coding. All Right Reserved", bg="white")
footer.config(font=('Courier', 11, 'bold'))
footer.place(x=x - 380, y=y + 260)

# disable the resize of the window
base.resizable(width=False, height=False)

''

In [11]:
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

In [12]:
ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

In [13]:
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

In [14]:
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
# EntryBox.bind("<Return>", send)

In [15]:
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=5, y=401, height=30, width=265)
SendButton.place(x=250, y=401, height=30)

In [16]:
base.mainloop()